# Práctica 2.

## Semana 1. Vecinos próximos

Read data

In [32]:
import numpy as np
import pandas as pd
from statistics import mode 

filename="ConjuntosDatosP2/pima-indians-diabetes.data"
filename2="ConjuntosDatosP2/wdbc.data"
data = pd.read_csv(filename)
data1=pd.read_csv(filename2)

column_names=list(data.columns)
column_names1=list(data1.columns)
print("Column names:")
print(column_names)
print(column_names1)
class_name="Class"

class_list=data[class_name].values
class_list1=data1[class_name].values

Column names:
['Pregs', 'Plas', 'Pres', 'Skin', 'Test', 'Mass', 'Pedi', 'Age', 'Class']
['Atributo1', 'Atributo2', 'Atributo3', 'Atributo4', 'Atributo5', 'Atributo6', 'Atributo7', 'Atributo8', 'Atributo9', 'Atributo10', 'Atributo11', 'Atributo12', 'Atributo13', 'Atributo14', 'Atributo15', 'Atributo16', 'Atributo17', 'Atributo18', 'Atributo19', 'Atributo20', 'Atributo21', 'Atributo22', 'Atributo23', 'Atributo24', 'Atributo25', 'Atributo26', 'Atributo27', 'Atributo28', 'Atributo29', 'Atributo30', 'Class']


In [46]:
import numpy as np
import pandas as pd
from statistics import mode 
class ClasificadorVecinosProximos:
    def __init__(self,train_set, classes):
        self.train=train_set.copy(deep=True)
        self.classes=classes.values
        self.class_names=np.unique(self.classes)
        self.normalize_done=False
        self.calcularMediasDesv()
        
    def calcularMediasDesv(self):
        self.mean=self.train.mean(axis=0)
        self.std=self.train.std(axis=0)
    def normalizarDato(self, dato):
        x=(dato-self.mean.values)/self.std.values
        return(x)
    def normalizarDatos(self):
        if not self.normalize_done:
            for att in self.train.columns:
                self.train[att]=(self.train[att]-self.mean[att])/self.std[att]
                self.normalize_done=True
        return(self.train)

    def euclidian_distance(self,dato):
        i=0
        square_dist=0
        for att in self.train.columns:
            square_dist=square_dist+(self.train[att].values-dato[i])**2
            i+=1
        square_dist=square_dist**0.5
        return(square_dist)
    def manhatten_distance(self,dato):
        i=0
        dist=0
        for att in self.train.columns:
            dist=dist+(self.train[att].values-dato[i])
            i+=1
        return(abs(dist))
    def mahalanobis_distance(self,dato): 
        X=self.train.values
        if self.normalize_done:
            mu=np.zeros(len(dato))
        else:
            mu=self.mean
        cov=np.cov(np.transpose(X-mu))
        diff=dato - mu
        inv=np.linalg.inv(cov)
        left=np.dot(diff,inv)
        dist=np.dot(left,np.transpose(diff))
        return(dist)
    def kneigbour_class(self, x, k, dist_type):
        if dist_type=="manhatten":
            dist=self.manhatten_distance(x)
        elif dist_type=="mahalanobis":
            dist=self.mahalanobis_distance(x)
        else:
            dist=self.euclidian_distance(x)
        idx = np.argpartition(dist, k)
        idx=idx[0:k]
        return mode(self.classes[idx])

In [34]:
import EstrategiaParticionado

estrategia=EstrategiaParticionado.ValidacionSimple(0.3, 1)
parts = estrategia.creaParticiones(data)
test_row_id=parts[0].indicesTest
train_row_id=parts[0].indicesTrain

atributes=['Pregs', 'Plas', 'Pres', 'Skin', 'Test', 'Mass', 'Pedi', 'Age']

#Selecting only columns with atributes required
train=data.loc[train_row_id,atributes].copy()
train_class_list=data.loc[train_row_id,class_name]


test=data.loc[test_row_id,atributes]
test_class_list=data.loc[test_row_id,class_name]

In [49]:
def error(model, test, test_classes, normalize, k, distance):
    err=0
    for index, row in test.iterrows():
        if normalize:
            row=model.normalizarDato(row.values)
        predicted=model.kneigbour_class(row, k, distance)
        if predicted!=test_classes.loc[index]:
            err+=1
    return (round(err/test.shape[0],2))

k_n=[1, 3, 5, 11, 21]


mod_kn=ClasificadorVecinosProximos(train, train_class_list)
norm_data=mod_kn.normalizarDatos()

print("Error:")
print("Dataset: ", filename)
print("Atributos utilizados: ", atributes)
print("Datos normalizados, distancia euclidia")
for k in k_n:
    print("K: ", k, error(mod_kn, test, test_class_list, True, k, "euclidian"))
print("Datos normalizados, distancia manhatten")
for k in k_n:
    print("K: ", k, error(mod_kn, test, test_class_list, True, k, "manhatten"))

    
  
mod=ClasificadorVecinosProximos(train, train_class_list)  
print("Datos sin normalizar, distancia euclidia")
for k in k_n:
    print("K: ", k, error(mod, test, test_class_list, False, k, "euclidian"))
print("Datos sin normalizar, distancia manhatten")
for k in k_n:
    print("K: ", k, error(mod, test, test_class_list, False, k, "manhatten"))

Error:
Dataset:  ConjuntosDatosP2/pima-indians-diabetes.data
Atributos utilizados:  ['Pregs', 'Plas', 'Pres', 'Skin', 'Test', 'Mass', 'Pedi', 'Age']
Datos normalizados, distancia euclidia
K:  1 0.3
K:  3 0.29
K:  5 0.3
K:  11 0.3
K:  21 0.27
Datos normalizados, distancia manhatten
K:  1 0.38
K:  3 0.33
K:  5 0.34
K:  11 0.32
K:  21 0.3
Datos sin normalizar, distancia euclidia
K:  1 0.34
K:  3 0.3
K:  5 0.29
K:  11 0.28
K:  21 0.3
Datos sin normalizar, distancia manhatten
K:  1 0.41
K:  3 0.39
K:  5 0.39
K:  11 0.36
K:  21 0.34


In [50]:
estrategia=EstrategiaParticionado.ValidacionSimple(0.3, 1)
parts1 = estrategia.creaParticiones(data1)
test_row_id1=parts1[0].indicesTest
train_row_id1=parts1[0].indicesTrain

atributes1=['Atributo1', 'Atributo2', 'Atributo3', 'Atributo4', 'Atributo5', 'Atributo6', 'Atributo7', 'Atributo8', 'Atributo9', 'Atributo10', 'Atributo11', 'Atributo12', 'Atributo13', 'Atributo14', 'Atributo15', 'Atributo16', 'Atributo17', 'Atributo18', 'Atributo19', 'Atributo20', 'Atributo21', 'Atributo22', 'Atributo23', 'Atributo24', 'Atributo25', 'Atributo26', 'Atributo27', 'Atributo28', 'Atributo29', 'Atributo30']

#Selecting only columns with atributes required
train1=data1.loc[train_row_id1,atributes1].copy()
train_class_list1=data1.loc[train_row_id1,class_name]


test1=data1.loc[test_row_id1,atributes1]
test_class_list1=data1.loc[test_row_id1,class_name]

In [51]:
mod_kn1=ClasificadorVecinosProximos(train1, train_class_list1)
norm_data1=mod_kn1.normalizarDatos()

print("Error:")
print("Dataset: ", filename2)
print("Atributos utilizados: ", atributes1)
print("Datos normalizados, distancia euclidia")
for k in k_n:
    print("K: ", k, error(mod_kn1, test1, test_class_list1, True, k, "euclidian"))
print("Datos normalizados, distancia manhatten")
for k in k_n:
    print("K: ", k, error(mod_kn1, test1, test_class_list1, True, k, "manhatten"))

    
mod1=ClasificadorVecinosProximos(train1, train_class_list1)  
print("Datos sin normalizar, distancia euclidia")
for k in k_n:
    print("K: ", k, error(mod1, test1, test_class_list1, False, k, "euclidian"))
print("Datos sin normalizar, distancia manhatten")
for k in k_n:
    print("K: ", k, error(mod1, test1, test_class_list1, False, k, "manhatten"))

Error:
Dataset:  ConjuntosDatosP2/wdbc.data
Atributos utilizados:  ['Atributo1', 'Atributo2', 'Atributo3', 'Atributo4', 'Atributo5', 'Atributo6', 'Atributo7', 'Atributo8', 'Atributo9', 'Atributo10', 'Atributo11', 'Atributo12', 'Atributo13', 'Atributo14', 'Atributo15', 'Atributo16', 'Atributo17', 'Atributo18', 'Atributo19', 'Atributo20', 'Atributo21', 'Atributo22', 'Atributo23', 'Atributo24', 'Atributo25', 'Atributo26', 'Atributo27', 'Atributo28', 'Atributo29', 'Atributo30']
Datos normalizados, distancia euclidia
K:  1 0.03
K:  3 0.02
K:  5 0.03
K:  11 0.02
K:  21 0.03
Datos normalizados, distancia manhatten
K:  1 0.17
K:  3 0.13
K:  5 0.13
K:  11 0.12
K:  21 0.12
Datos sin normalizar, distancia euclidia
K:  1 0.09
K:  3 0.06
K:  5 0.06
K:  11 0.09
K:  21 0.07
Datos sin normalizar, distancia manhatten
K:  1 0.13
K:  3 0.12
K:  5 0.11
K:  11 0.1
K:  21 0.11


In [71]:
from Datos import *
import Clasificador
print("Naive Bayes Error:")
dataset_nb=Datos(filename)

cl_nb=Clasificador.ClasificadorNaiveBayes(True)
errores=cl_nb.validacion(estrategia,dataset_nb,cl_nb)
print(filename, "error", round(errores[0],2))

dataset_nb1=Datos(filename2)

cl_nb1=Clasificador.ClasificadorNaiveBayes(True)
errores1=cl_nb1.validacion(estrategia,dataset_nb1,cl_nb1)
print(filename2, "error", round(errores1[0],2))

Naive Bayes Error:
ConjuntosDatosP2/pima-indians-diabetes.data error 0.22
ConjuntosDatosP2/wdbc.data error 0.11


In [58]:
def confusion_mx_knn(model, test, test_classes, normalize, k, distance, positive, negative):
    confusion_matrix=np.zeros([2,2])
    for index, row in test.iterrows():
        if normalize:
            row=model.normalizarDato(row.values)
        predicted=model.kneigbour_class(row, k, distance)
        real=test_classes.loc[index]
        if predicted==positive:
            if real==positive:
                confusion_matrix[0,0]+=1
            else: 
                confusion_matrix[0,1]+=1
        else:
            if real==negative:
                confusion_matrix[1,1]+=1
            else: 
                confusion_matrix[1,0]+=1

    return(confusion_matrix)

k=3
print("Matriz de confusión")
print("Dataset: ", filename)
print("Datos normalizados, distancia euclidia, k =", k)
print(confusion_mx_knn(mod_kn, test, test_class_list, True, k, "euclidian", 1, 0))
print("Datos sin normalizar, distancia euclidia, k =", k)
print(confusion_mx_knn(mod, test, test_class_list, False, k, "euclidian", 1, 0))

print("Dataset: ", filename2)

print("Datos normalizados, distancia euclidia, k =",k)
print(confusion_mx_knn(mod_kn1, test1, test_class_list1, True, k, "euclidian", "M", "B"))
print("Datos sin normalizar, distancia euclidia, k =", k)
print(confusion_mx_knn(mod1, test1, test_class_list1, False, k, "euclidian", "M", "B"))


Matriz de confusión
Dataset:  ConjuntosDatosP2/pima-indians-diabetes.data
Datos normalizados, distancia euclidia, k = 3
[[ 44.  24.]
 [ 42. 120.]]
Datos sin normalizar, distancia euclidia, k = 3
[[ 44.  27.]
 [ 42. 117.]]
Dataset:  ConjuntosDatosP2/wdbc.data
Datos normalizados, distancia euclidia, k = 3
[[ 51.   1.]
 [  3. 116.]]
Datos sin normalizar, distancia euclidia, k = 3
[[ 49.   6.]
 [  5. 111.]]


Test fields for calculating mahalanobis distance

In [87]:
from scipy.spatial import distance
arr1=test.loc[0].values
arr2=train.loc[1].values
m=np.matrix([arr1,arr2])
print(m)
Covariance = np.cov(m)
print(Covariance)
inv = np.linalg.inv(Covariance)
print(inv)
iv = [[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]
print(distance.mahalanobis([1, 0, 0], [0, 1, 0], iv))
distance.mahalanobis([0, 2, 0], [0, 1, 0], iv)
distance.mahalanobis([2, 0, 0], [0, 1, 0], iv)

[[  6.    148.     72.     35.      0.     33.6     0.627  50.   ]
 [  1.     85.     66.     29.      0.     26.6     0.351  31.   ]]
[[2440.09184827 1497.91970962]
 [1497.91970962  993.49795012]]
[[ 0.00550528 -0.00830044]
 [-0.00830044  0.0135213 ]]
1.0


1.7320508075688772

In [101]:
#create function to calculate Mahalanobis distance
def mahalanobis(x, y, cov=None):
    x_mean = np.mean(x)
    Covariance = np.cov(np.transpose(y))
    inv_covmat = np.linalg.inv(Covariance)
    x_minus_mn = x - y_mean
    D_square = np.dot(np.dot(x_minus_mn, inv_covmat), np.transpose(x_minus_mn))
    return D_square

#create new column in dataframe that contains Mahalanobis distance for each row
dist = mahalanobis(test.loc[0], train)
#display first five rows of dataframe
print(dist)

6.246646338576568


In [72]:
from scipy.spatial import distance
iv = [[1, 0.5, 0.5], [0.5, 1, 0.5], [0.5, 0.5, 1]]
print(distance.mahalanobis([1, 0, 0], [0, 1, 0], iv))
distance.mahalanobis([0, 2, 0], [0, 1, 0], iv)
distance.mahalanobis([2, 0, 0], [0, 1, 0], iv)

X=norm_data[atributes].values
mu=np.zeros(len(dato))
print(mu)
cov=np.dot(np.transpose(X),X)
print(np.cov(np.transpose(X)))
diff=dato - mu
print(diff)
inv=np.linalg.inv(cov)
left=np.dot(np.transpose(diff),inv)
dist=np.dot(left,diff)
print(cov)
print(dist)

1.0


NameError: name 'dato' is not defined

In [54]:
#for i in range(data.shape[0])

matrix=np.matrix(norm_data)
matrix=np.transpose(matrix)
example=matrix[column_ids]


X = np.stack((example, np.matrix(norm_point)), axis=0)
print(X)
cov=np.cov(X)
print(cov)
dist=(np.transpose(example - norm_point)*np.linalg.inv(cov)*(example - norm_point))**0.5

print(dist)

NameError: name 'column_ids' is not defined

## Semana 2. Regresión logistica
ClasificadorRegresionLogistica


Indians:

preg = Number of times pregnant

plas = Plasma glucose concentration a 2 hours in an oral glucose tolerance test

pres = Diastolic blood pressure (mm Hg)

skin = Triceps skin fold thickness (mm)

test = 2-Hour serum insulin (mu U/ml)

mass = Body mass index (weight in kg/(height in m)^2)

pedi = Diabetes pedigree function

age = Age (years)

class = Class variable (1:tested positive for diabetes, 0: tested negative for diabetes)

In [22]:
class ClasificadorRegresionLogistica:
    def __init__(self,train_set, classes):
        self.train_set=train_set
        self.classes=classes
        self.class_names=np.unique(self.classes)
        self.class1=self.class_names[0]
    def gradient_descent(self, rounds, eta):
        self.w=np.zeros(self.train_set.shape[1]+1)+0.5
        for r in range(rounds):
            #for i in range(self.train_set.shape[0]):
            for i, row in self.train_set.iterrows():
                #x=np.array(self.train_set.iloc[i].values)
                x=np.array(row.values)
                x=np.insert(x,0,1)
                x=np.transpose(x)
                if self.classes[i]==self.class1:
                    t=1
                else:
                    t=0
                wx=np.dot(self.w,x)
                self.w=self.w-eta*(1/(1+np.exp(-wx))-t)*x
        return(self.w)
    def classify(self,x):
        x=np.array(x)
        x=np.insert(x,0,1)
        x=np.transpose(x)
        wx=np.dot(self.w,x)
        if wx>=0:
            return self.class1
        else:
            return self.class_names[1]

In [23]:
atributes=["Pregs","Plas","Pres","Skin","Test"]
filter_zeros=[False, True, True, True, True]


#This one is not active now, but the data should be filtered previously as it contains 0 fields (first dataset)
#Filtering zero values
for i in range(len(atributes)):
    if filter_zeros[i]:
        data_filtered=data.loc[data[atributes[i]]!=0]
train_set=data_filtered[atributes]
classes=data_filtered[class_name].values


model_rg=ClasificadorRegresionLogistica(train,train_class_list)
w=model_rg.gradient_descent(10, 0.01)
print(w)
#print(test.iloc[1].values)
x=[6,148,72,35,0,33.6,0.627,50]
print(model_rg.classify(x))

[ 2.48462213 -3.2996518  -1.95167018  2.68811167  1.13201183 -2.27855089
  3.45473797  0.36901397  1.59798852]
0


In [28]:
def error_rg(model, test, test_classes):
    err=0
    for index, row in test.iterrows():
        predicted=model.classify(row)
        if predicted!=test_classes.loc[index]:
            err+=1
    return (round(err/test.shape[0],2))

model_rg=ClasificadorRegresionLogistica(train,train_class_list)
w=model_rg.gradient_descent(10, 0.01)
print("Error regresión logistica, dataset", filename)
print(error_rg(model_rg, test, test_class_list))
model_rg1=ClasificadorRegresionLogistica(train1,train_class_list1)
w=model_rg1.gradient_descent(10, 0.01)
print("Error regresión logistica, dataset", filename2)
print(error_rg(model_rg1, test1, test_class_list1))


Error regresión logistica, dataset ConjuntosDatosP2/pima-indians-diabetes.data
0.43
Error regresión logistica, dataset ConjuntosDatosP2/wdbc.data
0.09
